In [1]:
import pandas as pd
import numpy as np

In [3]:
train = pd.read_csv('/Users/caleboneel/Downloads/ultimate_train.csv')
val = pd.read_csv('/Users/caleboneel/Downloads/ultimate_val.csv')
test = pd.read_csv('/Users/caleboneel/Downloads/ultimate_test_us.csv')

In [7]:
frames = [train, val]
result = pd.concat(frames)
result

,log_price,accommodates,bathrooms,host_has_profile_pic,host_identity_verified,host_response_rate,instant_bookable,number_of_reviews,review_scores_rating,thumbnail_url,...,x0_Tipi,x0_Townhouse,x0_Train,x0_Treehouse,x0_Vacation home,x0_Villa,x0_Yurt,x1_Entire home/apt,x1_Private room,x1_Shared room
0,3.737670,1,1.0,1,0,100.0,0,0,91.0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,6.109248,5,2.0,1,0,100.0,0,1,100.0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,4.248495,2,1.0,1,1,90.0,0,68,95.0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,4.060443,1,1.0,1,0,25.0,0,0,98.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,4.369448,2,1.0,1,1,0.0,0,25,96.0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11853,4.248495,2,1.0,1,0,99.0,0,0,80.0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
11854,5.796058,2,1.0,1,1,87.0,0,0,100.0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
11855,4.605170,2,1.0,1,1,100.0,0,8,98.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
11856,5.389072,6,1.0,1,1,100.0,0,12,95.0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from time import time
import scipy.stats as stats
from sklearn.utils.fixes import loguniform
from tqdm import tqdm
from sklearn import linear_model
from sklearn.model_selection import RepeatedKFold
from numpy import absolute


# Create MLP model with desired parameters, omitting the ones we will use random search to find
clf = linear_model.Lasso()
kfold = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

# A function to print results of random search
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})"
                  .format(results['mean_test_score'][candidate],
                          results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")
            
# specify parameters and distributions to sample from
param_dist = {
              'alpha': loguniform(1e-10, 1e2),
           }

# run randomized search
n_iter_search = 50
random_search = RandomizedSearchCV(clf, param_distributions=param_dist, cv=kfold, n_jobs=-1,
                                   n_iter=n_iter_search, verbose=1).fit(X,y)

# Time and output results of random search model
start = time()
random_search.fit(X, y)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))
report(random_search.cv_results_)